In [ ]:
# for Collab notebooks, we will start by installing the ``collie_recs`` library
!pip install collie_recs --quiet

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

%env DATA_PATH data/

env: DATA_PATH=data/


# Partial Credit Loss

In [2]:
import os

import numpy as np
import pandas as pd
from pytorch_lightning.utilities.seed import seed_everything
from IPython.display import HTML
import joblib
import torch

from collie_recs.metrics import mapk, mrr, auc, evaluate_in_batches
from collie_recs.model import CollieTrainer, MatrixFactorizationModel
from collie_recs.movielens import get_movielens_metadata, get_recommendation_visualizations

Most of the time, we don't *only* have user-item interactions, but also side-data about our items that we are recommending. These next two notebooks will focus on incorporating this into the model training process. 

In this notebook, we're going to add a new component to our loss function - "partial credit". Specifically, we're going to use the genre information to give our model "partial credit" for predicting that a user would like a movie that they haven't interacted with, but is in the same genre as one that they liked. The goal is to help our model learn faster from these similarities. 

## Read in Data

In [3]:
# read in the data we saved from the first notebook
train_interactions = joblib.load(os.path.join(os.environ.get('DATA_PATH', 'data/'), 'train_interactions.pkl'))
val_interactions = joblib.load(os.path.join(os.environ.get('DATA_PATH', 'data/'), 'val_interactions.pkl'))

print('Train:', train_interactions)
print('Val:  ', val_interactions)

Train: Interactions object with 49426 interactions between 943 users and 1674 items, returning 10 negative samples per interaction.
Val:   Interactions object with 5949 interactions between 943 users and 1674 items, returning 10 negative samples per interaction.


In [4]:
# and the same metadata used in notebook 3
metadata_df = get_movielens_metadata()


metadata_df.head()

,genre_action,genre_adventure,genre_animation,genre_children,genre_comedy,genre_crime,genre_documentary,genre_drama,genre_fantasy,genre_film_noir,...,genre_unknown,decade_unknown,decade_20,decade_30,decade_40,decade_50,decade_60,decade_70,decade_80,decade_90
0,0,0,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [5]:
# to do the partial credit calculation, we need this data in a slightly different form.
# Instead of the one-hot-encoded version above, we're going to make a ``1 x n_items`` tensor
# with a number representing the first genre associated with the film, for simplicity.
# Note that with Collie, we could instead make a metadata tensor for each genre and decade
genres = (
    torch.tensor(metadata_df[[c for c in metadata_df.columns if 'genre' in c]].values)
    .topk(1)
    .indices
    .view(-1)
)


genres

tensor([ 2,  1, 15,  ..., 13,  4,  7])

In [6]:
# and, as always, set our random seed
seed_everything(22)

Global seed set to 22


22

## Train a model with our new loss

In [7]:
# now, we will pass in ``metadata_for_loss`` and ``metadata_for_loss_weights`` into the model
# ``metadata_for_loss`` should have a tensor containing the integer representations for metadata
#                       we created above for every item ID in our dataset
# ``metadata_for_loss_weights`` should have the weights for each of the keys in ``metadata_for_loss``
model = MatrixFactorizationModel(
    train=train_interactions,
    val=val_interactions,
    embedding_dim=10,
    lr=1e-2,
    metadata_for_loss={'genre': genres},
    metadata_for_loss_weights={'genre': 0.4},
)

In [8]:
trainer = CollieTrainer(model=model, max_epochs=10, deterministic=True)

trainer.fit(model)

GPU available: False, used: False
TPU available: None, using: 0 TPU cores

  | Name            | Type            | Params
----------------------------------------------------
0 | user_biases     | ZeroEmbedding   | 943   
1 | item_biases     | ZeroEmbedding   | 1.7 K 
2 | user_embeddings | ScaledEmbedding | 9.4 K 
3 | item_embeddings | ScaledEmbedding | 16.7 K
4 | dropout         | Dropout         | 0     
----------------------------------------------------
28.8 K    Trainable params
0         Non-trainable params
28.8 K    Total params
0.115     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Epoch     9: reducing learning rate of group 0 to 1.0000e-06.
Epoch     9: reducing learning rate of group 0 to 1.0000e-06.


Validating: 0it [00:00, ?it/s]

1

## Evaluate the Model 

Again, we'll evaluate the model and look at some particular users' recommendations to get a sense of what these recommendations look like using a partial credit loss function during model training. 

In [9]:
mapk_score, mrr_score, auc_score = evaluate_in_batches([mapk, mrr, auc], val_interactions, model)

print(f'MAP@10 Score: {mapk_score}')
print(f'MRR Score:    {mrr_score}')
print(f'AUC Score:    {auc_score}')

  0%|          | 0/48 [00:00<?, ?it/s]

MAP@10 Score: 0.052236116226391474
MRR Score:    0.16873707316989858
AUC Score:    0.907800395777271


Broken record alert: we're not seeing as much performance increase here compared to the model in Tutorial 02 because MovieLens 100K has so few items. For a more dramatic difference, try training this model on a larger dataset, such as MovieLens 10M, adjusting the architecture-specific hyperparameters, or train longer. 

In [10]:
user_id = np.random.randint(0, train_interactions.num_users)

display(
    HTML(
        get_recommendation_visualizations(
            model=model,
            user_id=user_id,
            filter_films=True,
            shuffle=True,
            detailed=True,
        )
    )
)

,Dances with Wolves (1990),Speed (1994),"Hunchback of Notre Dame, The (1996)",Cinderella (1950),Apollo 13 (1995),One Fine Day (1996),Indiana Jones and the Last Crusade (1989),Four Weddings and a Funeral (1994),"Truth About Cats & Dogs, The (1996)",Top Gun (1986)
Some loved films:,,,,,,,,,,
,Independence Day (ID4) (1996),"Empire Strikes Back, The (1980)",Return of the Jedi (1983),Star Wars (1977),Terminator 2: Judgment Day (1991),Braveheart (1995),"Hunt for Red October, The (1990)",Star Trek: First Contact (1996),Groundhog Day (1993),Star Trek: The Wrath of Khan (1982)
Recommended films:,,,,,,,,,,


Partial credit loss is useful when we want an easy way to boost performance of any implicit model architecture, hybrid or not. When tuned properly, partial credit loss more fairly penalizes the model for more egregious mistakes and relaxes the loss applied when items are more similar. 

Of course, the loss function isn't the only place we can incorporate this metadata - we can also directly use this in the model (and even use a hybrid model combined with partial credit loss). The next tutorial will train a hybrid Collie model! 

----- 